## I. Prerequesites

**1. Import Packages**

In [ ]:
import numpy as np
import pytensor.tensor as at
import pandas as pd
import pymc as pm
import arviz as az

**2. Define functions**

In [ ]:
def standardize(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize a pandas DataFrame"""
    standardized_df = pd.DataFrame()
    for column in df.columns:
        standardized_column = (df[column] - df[column].mean()) / df[column].std()
        standardized_df[column] = standardized_column
    return standardized_df

def standardize_series(series):
    """Standardize a pandas series"""
    return (series - series.mean()) / series.std()

def take_logarithm(df: pd.DataFrame) -> pd.DataFrame:
    """take logarithm a pandas DataFrame"""
    logarithmized_df = pd.DataFrame()
    for column in df.columns:
        logarithmized_column = np.log(df[column])
        logarithmized_df[column] = logarithmized_column
    return logarithmized_df

**3. Set style and generate random seed**

In [ ]:
az.style.use("arviz-darkgrid")
RANDOM_SEED = 58
rng = np.random.default_rng(RANDOM_SEED)

## II. Data Preparation

 **1. Load data**

In [ ]:
X = pd.read_parquet(r'C:\Users\Uwe Drauz\Documents\bachelor_thesis_local\personal_competition_data\temp\X.parquet')
y = pd.read_parquet(r'C:\Users\Uwe Drauz\Documents\bachelor_thesis_local\personal_competition_data\temp\y.parquet')
X = X.reset_index(drop=True, inplace=False)
y = y.reset_index(drop=True, inplace=False)

**2. Standardize data and/or take loagrithm of data**

In [ ]:
# Standardize target variable y
y_std = standardize(y)

# Calculate the standardized logarithm of input data X
X_std= standardize(X)
X_std_log = take_logarithm(X_std)

**3. Create data frames with standardized and/or logarithmized data**

In [ ]:
# Create data frame with target variable and standardized input data
df_Y_non_std_X_std = pd.concat([y, X_std], axis=1)
# Create data frame with standardized target variable and standardized input data
df_Y_std_X_Std = pd.concat([y_std, X_std], axis=1)
# Create data frame with target variable and standardized logarithm of input data
df_Y_non_std_X_std_log = pd.concat([y, X_std_log], axis=1)
# Create data frame with standardized target variable and standardized logarithm of input data
df_Y_std_X_std_log = pd.concat([y_std, X_std_log], axis=1)

**4. Define which data to use**

In [ ]:
df = df_Y_non_std_X_std

## III. Model Building

In [ ]:
# Define dictionary for implicit handling of data
COORDS = {"predictors": ['ged_sb_tlag_1', 'ged_sb_tsum_24'], "obs_idx": df.index}

**1. Build Bayesian model with Gaussian distribution**

In [ ]:
with pm.Model(coords=COORDS) as bayesian_model_gauss:
    # Define the priors for regressors and standard deviation
    intercept = pm.Normal('intercept', 0.0, 1.0)
    beta = pm.Normal("slopes", 0.0, 1.0, dims="predictors")
    sigma = pm.HalfNormal("sigma", 25)

    # Specify the data
    X1 = pm.ConstantData("fatality lag t=1", df.ged_sb_tlag_1.to_numpy(), dims="obs_idx")
    X2 = pm.ConstantData("fatality rolling average t=24", df.ged_sb_tsum_24.to_numpy(), dims="obs_idx")
    Y = pm.ConstantData("fatility count", df.ged_sb.to_numpy(), dims="obs_idx")

    # Specify mean of gaussian distribution
    mu = intercept +  beta[0] * X1 + beta[1] * X2
    # mean for target variable
    obs = pm.Normal("obs", mu=mu, sigma=sigma, observed=Y, dims="obs_idx")

    # Run the sampling using the No-U-Turn Sampler (NUTS) for the specified number of samples
    idata_bayesian_gauss = pm.sample(1000, tune=2000, random_seed=rng)

In [ ]:
with bayesian_model_gauss:
    ppc_prior = pm.sample_prior_predictive(samples=500, random_seed=rng)
    pm.sample_posterior_predictive(idata_bayesian_gauss, extend_inferencedata=True, random_seed=rng)

**2. Build Bayesian model with Negative Binomial distribution**

In [ ]:
# Build the model
with pm.Model(coords=COORDS) as bayesian_model_nb:
    # Define the priors for regressors and  negative binomial over-dispersion parameter
    intercept = pm.HalfNormal("intercept", sigma=0.1)
    beta = pm.Normal("slopes", 0.0, 10, dims="predictors")
    alpha = pm.HalfNormal('alpha', 2.5)

    # Specify the data
    X1 = pm.ConstantData("fatality lag t=1", df.ged_sb_tlag_1.to_numpy(), dims="obs_idx")
    X2 = pm.ConstantData("fatality rolling average t=24", df.ged_sb_tsum_24.to_numpy(), dims="obs_idx")
    Y = pm.ConstantData("fatility count", df.ged_sb.to_numpy(), dims="obs_idx")

    # Note: Possibly an option to change to pm.Deterministic
    # Define mean of negative binomial distribution
    mu = pm.math.exp(intercept + beta[0] * X1 + beta[1] * X2)

    # Define the likelihood
    obs = pm.NegativeBinomial("obs", mu=mu, alpha=alpha, observed=Y, dims="obs_idx")

    # Note: Optionally define cores and chains
    # Run the sampling using the No-U-Turn Sampler (NUTS) for the specified number of samples
    idata_bayesian_nb = pm.sample(draws=1000, tune=2000, random_seed=rng, target_accept=0.95)

In [ ]:
with bayesian_model_nb:
    ppc_prior = pm.sample_prior_predictive(samples=500, random_seed=rng)
    ppc_posterior = pm.sample_posterior_predictive(idata_bayesian_nb, extend_inferencedata=True, random_seed=rng)

## IV. Model Evaluation

**tbd: what exactly do we examine here?**

**Analysis for negative binomial model**

In [ ]:
# Specify which model should be examined
model = bayesian_model_nb
idata = idata_bayesian_nb


In [ ]:
az.plot_trace(idata)
az.plot_trace(idata, combined=True)
az.plot_posterior(idata)

In [ ]:
# Transform coefficients to recover parameter values
az.summary(np.exp(idata.posterior), kind="stats", var_names=["intercept", "slopes"])

In [ ]:
az.summary(idata.posterior, kind="stats", var_names="alpha")

In [ ]:
az.plot_ppc(idata, num_pp_samples=100)

**Analysis for Gaussian model**

In [ ]:
# Specify which model should be examined
model = bayesian_model_gauss
idata = idata_bayesian_gauss

In [ ]:
az.plot_trace(idata)
az.plot_trace(idata, combined=True)
az.plot_posterior(idata)

In [ ]:
# Transform coefficients to recover parameter values
az.summary(np.exp(idata.posterior), kind="stats", var_names=["intercept", "slopes"])

In [ ]:
az.summary(idata.posterior, kind="stats", var_names="sigma")

In [ ]:
az.plot_ppc(idata, num_pp_samples=100)

## V. Test functionalities with simulated data

In [ ]:
N = 100

true_a, true_b, predictor = 0.5, 3.0, rng.normal(loc=2, scale=6, size=N)
true_mu = true_a + true_b * predictor
true_sd = 2.0

outcome = rng.normal(loc=true_mu, scale=true_sd, size=N)

In [ ]:
predictor_scaled = standardize_series(predictor)
outcome_scaled = standardize_series(outcome)

In [ ]:
with pm.Model() as model_1:
    a = pm.Normal("a", 0.0, 0.5)
    b = pm.Normal("b", 0.0, 1.0)

    mu = a + b * predictor_scaled
    sigma = pm.Exponential("sigma", 1.0)

    pm.Normal("obs", mu=mu, sigma=sigma, observed=outcome_scaled)
    idata = pm.sample_prior_predictive(samples=50, random_seed=rng)

In [ ]:
with model_1:
    idata.extend(pm.sample(1000, tune=2000, random_seed=rng))

az.plot_trace(idata);

In [ ]:
with model_1:
    pm.sample_posterior_predictive(idata, extend_inferencedata=True, random_seed=rng)

In [ ]:
az.plot_ppc(idata, num_pp_samples=100)

## VI. Data exploration steps

In [ ]:
predictor_scaled.shape

In [ ]:
outcome_scaled.shape


In [ ]:
y

In [ ]:
alpha

In [ ]:
intercept

In [ ]:
mu

In [ ]:
η

In [ ]:
obs

In [ ]:
alpha/(mu + alpha)